# Packages

In [ ]:
import cv2
import numpy as np

# Window settings for webcam

In [ ]:
import cv2

frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, 150)
while True:
    success, img = cap.read()
    cv2.imshow("Result", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

False

# Basic functions

In [ ]:
img = cv2.imread('cat.jpg')

#crop image
imgCropped = img[0:500,500:1000]

#img resize
height = int(img.shape[0]/3.5)
width = int(img.shape[1]/3.5)
imgResize = cv2.resize(img, (width, height)) #width first

imgGray = cv2.cvtColor(imgResize, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgResize, (1,1), 0) # kernel = positive and odd. Or, zero's and they are computed from sigma.
imgCanny = cv2.Canny(imgResize, 200, 200) #edges

kernel = np.ones((2,2), np.uint8)
imgDialation = cv2.dilate(imgCanny, kernel, iterations=1) #thickness
imgEroded = cv2.erode(imgDialation, kernel, iterations=1) #oposite of dialation
                     
#cv2.imshow('imgGray', imgGray)
cv2.waitKey(0)

-1

# Drawing shape

In [ ]:

img = np.zeros((512,512,3),np.uint8)
#print(img)
#img[:]= 255,0,0
 
cv2.line(img,(0,0),(img.shape[1],img.shape[0]),(0,255,0),3)
cv2.rectangle(img,(0,0),(250,350),(0,0,255),2)
cv2.circle(img,(400,50),30,(255,255,0),5)
cv2.putText(img," OPENCV  ",(300,200),cv2.FONT_HERSHEY_COMPLEX,1,(0,150,0),3)
 
#cv2.imshow("Image",img)
cv2.waitKey(0)

-1

# Bird eye view ------Perspective

In [ ]:
img = cv2.imread('card.jpg')

def perspective(approx, img):
    #approx = perspectiveReorder(approx)
    width, height = 250, 350
    #pts1 = np.float32(approx)
    pts1 = np.float32([[194,143],[390,164],[113,363],[343,394]])
    pts2 = np.float32([[0,0],[width, 0], [0, height],[width,height]])

    mat = cv2.getPerspectiveTransform(pts1, pts2)
    imgCroped = cv2.warpPerspective(img, mat, (width, height))

    return imgCroped

def perspectiveReorder(myPoints):
    myPoints = myPoints.reshape((4,2))
    myPointsNew = np.zeros((4,1,2),np.int32)
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    #print(myPointsNew)
    return myPointsNew

x = perspective(np.nan, img)
cv2.imshow('a', x)

# Image Stack

In [ ]:
img = cv2.imread('cat.jpg')

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver
 
img = cv2.imread('cat1.jpg')
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

imgStack = stackImages(0.5,([img,imgGray,img],[img,img,img]))

#cv2.imshow('stacked image', imgStack)
cv2.waitKey(0)

-1

# CONTOURS

In [ ]:
def getContours(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for _ in contours:
        area = cv2.contourArea(_)
        #print(area)
        if area>10:
            cv2.drawContours(imgContour, _, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(_,True)    # curve length
            #print(peri)
            approx = cv2.approxPolyDP(_,0.02*peri,True) # corner point
            #print(approx)
            cornerPoints = len(approx)

            x, y, w, h = cv2.boundingRect(approx)
            #print(x, y, w, h)
            cv2.rectangle(imgContour, (x,y), (x+w, y+h), (0,255,0), 3)
            #cv2.imshow('sleep', imgContour)
            #cv2.waitKey(1)
            #time.sleep(2)

            if cornerPoints == 3: objectType = 'trangle'
            else: objectType = None
            cv2.putText(imgContour,objectType,
                        (x+(w//2)-10,y+(h//2)-10),cv2.FONT_HERSHEY_COMPLEX,0.7,
                        (0,0,0),2)

 
img = cv2.imread('shape.png')
imgContour = img.copy()
imgBlank = np.zeros_like(img)
 
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray,(7,7),1)
imgCanny = cv2.Canny(imgBlur,50,50)
getContours(imgCanny)
 
imgStack = stackImages(0.5,([img,imgGray,imgBlur],
                            [imgCanny,imgContour,imgBlank]))
 
# cv2.imshow("Stack", imgStack)
cv2.waitKey(0)

-1

# TrackBar

In [ ]:
def TrackBar():
    cv2.namedWindow("TrackBars")
    cv2.resizeWindow("TrackBars",640,320)
    cv2.createTrackbar("Hue Min","TrackBars",0,179,lambda a: a)
    cv2.createTrackbar("Hue Max","TrackBars",0,179,lambda a: a)
    cv2.createTrackbar("Sat Min","TrackBars",195,255,lambda a: a)
    cv2.createTrackbar("Sat Max","TrackBars",255,255,lambda a: a)
    cv2.createTrackbar("Val Min","TrackBars",0,255,lambda a: a)
    cv2.createTrackbar("Val Max","TrackBars",255,255,lambda a: a)

    while True:
        h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
        h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
        s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
        s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
        v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
        v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
        print(h_min,h_max,s_min,s_max,v_min,v_max)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

#TrackBar()

# ColorDetector HSV

In [ ]:
def ColorDetector():
    cv2.namedWindow("TrackBars")
    cv2.resizeWindow("TrackBars",640,320)
    cv2.createTrackbar("Hue Min","TrackBars",0,179,lambda a: a)
    cv2.createTrackbar("Hue Max","TrackBars",0,179,lambda a: a)
    cv2.createTrackbar("Sat Min","TrackBars",195,255,lambda a: a)
    cv2.createTrackbar("Sat Max","TrackBars",255,255,lambda a: a)
    cv2.createTrackbar("Val Min","TrackBars",0,255,lambda a: a)
    cv2.createTrackbar("Val Max","TrackBars",255,255,lambda a: a)

    while True:
        img = cv2.imread('color.png')
        imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
        h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
        s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
        s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
        v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
        v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
        print(h_min,h_max,s_min,s_max,v_min,v_max)
        lower = np.array([h_min,s_min,v_min])
        upper = np.array([h_max,s_max,v_max])
        mask = cv2.inRange(imgHSV,lower,upper)
        imgResult = cv2.bitwise_and(img,img,mask=mask)
    
        imgStack = stackImages(.3,([img,imgHSV],[mask,imgResult]))
        cv2.imshow("Stacked Images", imgStack)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

#ColorDetector()

# ColorDetector RBG

In [ ]:
def ColorDetector():
    cv2.namedWindow("TrackBars")
    cv2.resizeWindow("TrackBars",640,320)
    cv2.createTrackbar("Red Min","TrackBars",0,255,lambda a: a)
    cv2.createTrackbar("Red Max","TrackBars",255,255,lambda a: a)
    cv2.createTrackbar("Green Min","TrackBars",0,255,lambda a: a)
    cv2.createTrackbar("Green Max","TrackBars",255,255,lambda a: a)
    cv2.createTrackbar("Blue Min","TrackBars",0,255,lambda a: a)
    cv2.createTrackbar("Blue Max","TrackBars",255,255,lambda a: a)

    while True:
        img = cv2.imread('color.png')
        imgRgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        h_min = cv2.getTrackbarPos("Red Min","TrackBars")
        h_max = cv2.getTrackbarPos("Red Max", "TrackBars")
        s_min = cv2.getTrackbarPos("Green Min", "TrackBars")
        s_max = cv2.getTrackbarPos("Green Max", "TrackBars")
        v_min = cv2.getTrackbarPos("Blue Min", "TrackBars")
        v_max = cv2.getTrackbarPos("Blue Max", "TrackBars")
        print(h_min,h_max,s_min,s_max,v_min,v_max)
        lower = np.array([h_min,s_min,v_min])
        upper = np.array([h_max,s_max,v_max])
        mask = cv2.inRange(imgRgb,lower,upper)
        imgResult = cv2.bitwise_and(img,img,mask=mask)

        imgStack = stackImages(.3,([img,imgRgb],[mask,imgResult]))
        cv2.imshow("Stacked Images", imgStack)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

#ColorDetector()

# Template Matching

In [ ]:
def tempMatch():
    img_rgb = cv2.imread('') # Main image
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

    template = cv2.imread('',0)
    w, h = template.shape[::-1]

    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    loc = np.where( res >= threshold)

    for pt in zip(*loc[::-1]):
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,255,255), 2)

    cv2.imshow('Detected',img_rgb)